In [1]:
%matplotlib widget

In [2]:
import aware

AWARE config file not found, using the default settings


## Provide the path to the actual catalog FITS table

In [3]:
aware.config.set_config_option_value("glade.main.catalog_path", "/mnt/e/Catalogs/GLADE_Plus/glade.fits")

In [4]:
import os
import asyncio
from dataclasses import dataclass, field
from textwrap import dedent
import aware
from datetime import datetime, timedelta
import os
from pprint import pprint
import sys
from typing import Sequence, Union
from astropy.coordinates import EarthLocation, SkyCoord
from astroplan.observer import Observer
from astroplan import FixedTarget
import astropy.units as u
from astropy.time import Time, TimeDelta
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from pathvalidate import sanitize_filepath
from aware.sql.models import create_session
from aware.alert.target_info import TargetInfo
from ligo.skymap.io.fits import read_sky_map
from ligo.skymap.moc import rasterize
from aware.localization.main import LVCSkyMap
from aware import site
from aware.planning.program import create_observation_program
from aware.consumer.main import save_file
from aware.logger import log
import click
import asyncclick as click
import aiofiles
from astropy.coordinates import EarthLocation, AltAz
from collections import defaultdict
import pickle
import aware
from ligo.skymap.plot import ScaleBar
from ligo.skymap.io.fits import write_sky_map
from ligo.skymap.postprocess.util import find_greedy_credible_levels
from ligo.skymap.postprocess.contour import contour
from ligo.skymap.postprocess.crossmatch import crossmatch
import matplotlib as mpl
from astroplan.scheduling import PriorityScheduler, ObservingBlock, Slot, Scorer, Schedule
from astroplan.constraints import MoonSeparationConstraint, SunSeparationConstraint
import healpy as hp
from aware.glade import GladeCatalog, GladeGalaxy
from matplotlib.patches import PathPatch, Path, Arrow, FancyArrow
from aware.planning.distributor import TargetDistributor, FieldDistributor, _get_target_hpx_id, _targets_per_night, paint_planned_healpix_from_galaxies
from aware.planning.program import create_observation_program
from aware.angle import coord_to_target_name
import gc
import re
import glob
from aware.planning.planner import SkymapPlanner
from astropy.io import fits

In [5]:
url = "https://gracedb.ligo.org/api/superevents/S230528a/files/bayestar.multiorder.fits,0"

In [6]:
sky_map = read_sky_map(url, nest=True, moc=True)

In [7]:
event_name = re.findall("S\d{6}[a-zA-Z]+", url)[0]
event_name

'S230528a'

In [8]:
hdr = sky_map.meta
data = sky_map

uniq = data["UNIQ"]
probdensity = data["PROBDENSITY"]
distmu = hdr["distmean"] * u.Mpc
distsigma = hdr["diststd"] * u.Mpc

loc = LVCSkyMap(
    uniq=uniq,
    data=data,
    probdensity=probdensity,
    distmu=distmu,
    distsigma=distsigma,
)

info = TargetInfo(loc, "", "")

In [9]:
TargetInfo.event

''

In [10]:
wide_field_telescopes=["abao_as32", "kitab_rc36", "altai_santel400", "uafo_rc500"]
narrow_field_telescopes=["assy_azt20", "sao_ras_zeiss1000", "tshao_zeiss1000"]

In [11]:
p = SkymapPlanner(sky_map)

In [12]:
p.event_hpx

array([8.85543007e-09, 8.85543007e-09, 8.85543007e-09, ...,
       2.92132307e-12, 2.92132307e-12, 2.92132307e-12])

In [13]:
p.plan_observations(wide_field_telescopes=wide_field_telescopes, 
                    narrow_field_telescopes=narrow_field_telescopes,
                    disable_intersections=False)

/home/halo/aware_dev/lib/python3.10/site-packages/erfa/core.py:154: ErfaWarning: ERFA function "utctai" yielded 1 of "dubious year (Note 3)"
  warnings.warn('ERFA function "{}" yielded {}'.format(func_name, wmsg),
/home/halo/aware_dev/lib/python3.10/site-packages/erfa/core.py:154: ErfaWarning: ERFA function "d2dtf" yielded 1 of "dubious year (Note 5)"
  warnings.warn('ERFA function "{}" yielded {}'.format(func_name, wmsg),


array([8.85543007e-09, 8.85543007e-09, 8.85543007e-09, ...,
       2.92132307e-12, 2.92132307e-12, 2.92132307e-12])

In [14]:
p.save_blocks()

2023-06-11 23:34:43,378.378  WARNING planner._save_block(): Observation program for AbAO AS-32 is empty
2023-06-11 23:34:43,398.398  WARNING planner._save_block(): Observation program for Kitab-ISON RC-36 is empty
2023-06-11 23:34:43,406.406  WARNING planner._save_block(): Observation program for Multa Santel-400 is empty
2023-06-11 23:34:43,416.416  WARNING planner._save_block(): Observation program for Assy-Turgen AZT-20 is empty
2023-06-11 23:34:43,427.427  WARNING planner._save_block(): Observation program for SAO RAS Zeiss-1000 is empty
2023-06-11 23:34:43,438.438  WARNING planner._save_block(): Observation program for TSHAO Zeiss-1000 is empty


{'abao_as32': '/mnt/e/Repositories/aware-repo/playground/abao_as32/fields__abao_as32_day1.txt',
 'kitab_rc36': '/mnt/e/Repositories/aware-repo/playground/kitab_rc36/fields__kitab_rc36_day1.objlist',
 'altai_santel400': '/mnt/e/Repositories/aware-repo/playground/altai_santel400/fields__altai_santel400_day1.objlist',
 'assy_azt20': '/mnt/e/Repositories/aware-repo/playground/assy_azt20/galaxies__assy_azt20_day1.txt',
 'sao_ras_zeiss1000': '/mnt/e/Repositories/aware-repo/playground/sao_ras_zeiss1000/galaxies__sao_ras_zeiss1000_day1.txt',
 'tshao_zeiss1000': '/mnt/e/Repositories/aware-repo/playground/tshao_zeiss1000/galaxies__tshao_zeiss1000_day1.txt'}

In [15]:
p.save_plan_fits()

In [16]:
p.day

1

In [17]:
p.reset()

UFuncTypeError: ufunc 'multiply' did not contain a loop with signature matching types (dtype([('PROBDENSITY', '<f8'), ('DISTMU', '<f8'), ('DISTSIGMA', '<f8'), ('DISTNORM', '<f8')]), dtype('float64')) -> None

In [ ]:
hpx = p.event_hpx["PROBDENSITY"]
nside = hp.get_nside(hpx)
area = hp.nside2pixarea(nside)
hpx * area

In [18]:
p.day

1